# Projeto 1 - Ciência dos Dados

Nome: Adney Costa Moura

Nome: Lister Ogusuku Ribeiro

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [27]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [28]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\adney\OneDrive\Documentos\2° semestre - Insper\Ciência dos Dados\ProjetoCdados_AdneyLister


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [29]:
filename = 'nubank.xlsx'

In [30]:
train = pd.read_excel(filename)
train.head(5)

Treinamento  Relevância
0                  @nubank você me ilude tão bem 🤩🤩🤩           0
1  vou ter que fazer parcelamento da parcela, eu ...           0
2  @hs_ramao @nubank aumentaram o meu e eu nem us...           1
3  hoje eu mal acordei e tava pegando ônibus o re...           0
4   🤣🤣🤣 @nubank vamos amigar https://t.co/7k6syhkxf0           0

In [31]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

Teste  Relevância
0  @ogustamlk gu, obrigado por nos avisar sobre i...           0
1  @nubank nubank libera logo meu cartao de crédi...           1
2  nubank foi o melhor cartão que eu já fiz até h...           1
3                            @nubank quer casar cmg?           0
4         aeeee carai, nubank aceitou meu crédito :3           1

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto foi a startup de serviços financeiros Nubank. Consideramos como relevante as operações que envolvem o limite de crédito, interação com o cliente e reclamações.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [48]:
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [49]:
# Separando cada row do data frame, fazendo um split das palavras e limpando cada frase. Além disso foi filtrado cada row
# as listas de relevância.
lista_relevante = []
lista_irrelevante = []

for i in range(0,500):
    # Separando os rows do DataFrame
    x = cleanup(train.Treinamento[i]).lower().split()
 
    if train.iloc[i,1] == 1:
        j = 0
        while j < len(x):
            lista_relevante.append(x[j])
            j+=1
        
    else:
        k = 0
        while k < len(x):
            lista_irrelevante.append(x[k])
            k+=1

# Tweets Relevantes

In [50]:
# Guardando as palavras como um pd.Series
serie_relevante = pd.Series(lista_relevante)
serie_relevante.head()

0     @hs_ramao
1       @nubank
2    aumentaram
3             o
4           meu
dtype: object

##### Frequências Absolutas

In [51]:
tabela_relevante_absoluta = serie_relevante.value_counts()
tabela_relevante_absoluta

@nubank    135
o          117
nubank      97
de          95
meu         94
          ... 
roupa        1
mesmos       1
tmbm         1
certeza      1
/´¯`•¸       1
Length: 1091, dtype: int64

##### Frequências Relativas

In [38]:
tabela_relevante_relativa = serie_relevante.value_counts(True)
tabela_relevante_relativa

@nubank    0.038516
o          0.033381
nubank     0.027675
de         0.027104
meu        0.026819
             ...   
roupa      0.000285
mesmos     0.000285
tmbm       0.000285
certeza    0.000285
/´¯`•¸     0.000285
Length: 1091, dtype: float64

# Tweets Irrelevantes

In [39]:
# Guardando as palavras como um pd.Series
serie_irrelevante = pd.Series(lista_irrelevante)
serie_irrelevante.head()

0    @nubank
1       você
2         me
3      ilude
4        tão
dtype: object

##### Frequências Absolutas

In [40]:
tabela_irrelevante_absoluta = serie_irrelevante.value_counts()
tabela_irrelevante_absoluta

de                       132
@nubank                  125
o                        116
que                      109
a                        106
                        ... 
https//tco/ddkqjxlv0f      1
@out_pex                   1
seus                       1
aumentarem                 1
crypto                     1
Length: 1780, dtype: int64

##### Frequências Relativas

In [41]:
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)
tabela_irrelevante_relativa

de                       0.025414
@nubank                  0.024066
o                        0.022333
que                      0.020986
a                        0.020408
                           ...   
https//tco/ddkqjxlv0f    0.000193
@out_pex                 0.000193
seus                     0.000193
aumentarem               0.000193
crypto                   0.000193
Length: 1780, dtype: float64

# Tweets Totais

In [42]:
total = lista_relevante + lista_irrelevante

In [43]:
total_series = pd.Series(total)

##### Frequências Absolutas

In [44]:
tabela_total = total_series.value_counts()
tabela_total

@nubank    260
o          233
de         227
nubank     191
que        166
          ... 
podiase      1
agencia      1
mó           1
with         1
crypto       1
Length: 2392, dtype: int64

##### Frequências Relativas

In [77]:
tabela_total_relativa = total_series.value_counts(True)
tabela_total_relativa

@nubank    0.029888
o          0.026785
de         0.026095
nubank     0.021957
que        0.019083
             ...   
podiase    0.000115
agencia    0.000115
mó         0.000115
with       0.000115
crypto     0.000115
Length: 2392, dtype: float64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [79]:
lista_test = []
lista_relevante_test = []
lista_irrelevante_test = []

for i in range(0,300):
    x = cleanup(test.Teste[i]).lower().split()
    
    lista_test.append(x)
    
    if train.iloc[i,1] == 1:
        j = 0
        while j < len(x):
            lista_relevante_test.append(x[j])
            j+=1
        
    else:
        k = 0
        while k < len(x):
            lista_irrelevante_test.append(x[k])
            k+=1

In [200]:
contador = 0
Pr = len(lista_relevante)/len(total)
Pi = len(lista_irrelevante)/len(total)
Prob_d_ser_relevante = []
Prob_d_ser_irrelevante = []

while contador < len(lista_test):
    
    acumulador_relevante = 1
    acumulador_irrelevante = 1

    for h in lista_test[contador]:

        if h not in lista_relevante:
            tabela_relevante_absoluta[h] = 0

        if h not in lista_irrelevante:
            tabela_irrelevante_absoluta[h] = 0

        acumulador_relevante *= ( tabela_relevante_absoluta[h] + 1 ) / ( len(lista_relevante) + len(total) )
        acumulador_irrelevante *= ( tabela_irrelevante_absoluta[h] + 1 ) / ( len(lista_irrelevante) + len(total) ) 
        #print('r:{0}  i:{1}'.format(acumulador_relevante,acumulador_irrelevante))
        
    # Probabilidades
#     Prob_d_ser_relevante = Pr*acumulador_relevante
#     Prob_d_ser_irrelevante = Pi*acumulador_irrelevante
    Prob_d_ser_relevante.append(Pr*acumulador_relevante)
    Prob_d_ser_irrelevante.append(Pi*acumulador_irrelevante)

    print('r:{0}  i:{1}'.format(Prob_d_ser_relevante,Prob_d_ser_irrelevante))


    if Prob_d_ser_relevante[contador] > Prob_d_ser_irrelevante[contador]:
        test.loc[contador,["Relevância_Classificador"]] = 1

    else:
        test.loc[contador,["Relevância_Classificador"]] = 0
        
    contador = contador + 1

r:[2.7220974239371965e-121]  i:[4.691018264649167e-115]
r:[2.7220974239371965e-121, 6.668472836172885e-45]  i:[4.691018264649167e-115, 8.99027607345295e-47]
r:[2.7220974239371965e-121, 6.668472836172885e-45, 4.408174156171615e-56]  i:[4.691018264649167e-115, 8.99027607345295e-47, 1.0968418003657567e-55]
r:[2.7220974239371965e-121, 6.668472836172885e-45, 4.408174156171615e-56, 2.2232654211962585e-14]  i:[4.691018264649167e-115, 8.99027607345295e-47, 1.0968418003657567e-55, 1.2116306951176556e-14]
r:[2.7220974239371965e-121, 6.668472836172885e-45, 4.408174156171615e-56, 2.2232654211962585e-14, 1.116437011887459e-23]  i:[4.691018264649167e-115, 8.99027607345295e-47, 1.0968418003657567e-55, 1.2116306951176556e-14, 5.995810356418477e-25]
r:[2.7220974239371965e-121, 6.668472836172885e-45, 4.408174156171615e-56, 2.2232654211962585e-14, 1.116437011887459e-23, 2.2631709208652925e-97]  i:[4.691018264649167e-115, 8.99027607345295e-47, 1.0968418003657567e-55, 1.2116306951176556e-14, 5.995810356418

r:[2.7220974239371965e-121, 6.668472836172885e-45, 4.408174156171615e-56, 2.2232654211962585e-14, 1.116437011887459e-23, 2.2631709208652925e-97, 4.001254561156886e-73, 3.444446746083553e-76, 1.1065004398429613e-25, 1.9854885802031218e-152, 2.7058390409879655e-77, 1.6279638720167485e-09, 2.5110807515222447e-39, 2.615973174090364e-76, 3.190183773221097e-130, 1.4883589953874134e-20, 1.6368437700683431e-177, 5.1144567112991823e-23, 7.148464160481343e-71, 1.0120841168634412e-18, 1.9182415101006465e-43, 3.4088797651172235e-28, 4.377861288250091e-70, 8.596909856647964e-18, 3.1163377744960457e-65, 1.7003508959730784e-34, 6.478765296305554e-67, 1.891905972958342e-101, 1.139637438712757e-113, 8.48423912134661e-25, 7.470472073199248e-32, 1.4330065884610723e-38, 2.0178509043623295e-78, 7.762422313879849e-42, 1.1130262979120835e-25, 1.7294352807594737e-29, 5.081257795564035e-144, 4.604799527234278e-22, 5.780804644994604e-105, 1.0687608274077939e-16, 5.732975276393661e-79, 1.238110781902637e-39, 2.7

r:[2.7220974239371965e-121, 6.668472836172885e-45, 4.408174156171615e-56, 2.2232654211962585e-14, 1.116437011887459e-23, 2.2631709208652925e-97, 4.001254561156886e-73, 3.444446746083553e-76, 1.1065004398429613e-25, 1.9854885802031218e-152, 2.7058390409879655e-77, 1.6279638720167485e-09, 2.5110807515222447e-39, 2.615973174090364e-76, 3.190183773221097e-130, 1.4883589953874134e-20, 1.6368437700683431e-177, 5.1144567112991823e-23, 7.148464160481343e-71, 1.0120841168634412e-18, 1.9182415101006465e-43, 3.4088797651172235e-28, 4.377861288250091e-70, 8.596909856647964e-18, 3.1163377744960457e-65, 1.7003508959730784e-34, 6.478765296305554e-67, 1.891905972958342e-101, 1.139637438712757e-113, 8.48423912134661e-25, 7.470472073199248e-32, 1.4330065884610723e-38, 2.0178509043623295e-78, 7.762422313879849e-42, 1.1130262979120835e-25, 1.7294352807594737e-29, 5.081257795564035e-144, 4.604799527234278e-22, 5.780804644994604e-105, 1.0687608274077939e-16, 5.732975276393661e-79, 1.238110781902637e-39, 2.7

r:[2.7220974239371965e-121, 6.668472836172885e-45, 4.408174156171615e-56, 2.2232654211962585e-14, 1.116437011887459e-23, 2.2631709208652925e-97, 4.001254561156886e-73, 3.444446746083553e-76, 1.1065004398429613e-25, 1.9854885802031218e-152, 2.7058390409879655e-77, 1.6279638720167485e-09, 2.5110807515222447e-39, 2.615973174090364e-76, 3.190183773221097e-130, 1.4883589953874134e-20, 1.6368437700683431e-177, 5.1144567112991823e-23, 7.148464160481343e-71, 1.0120841168634412e-18, 1.9182415101006465e-43, 3.4088797651172235e-28, 4.377861288250091e-70, 8.596909856647964e-18, 3.1163377744960457e-65, 1.7003508959730784e-34, 6.478765296305554e-67, 1.891905972958342e-101, 1.139637438712757e-113, 8.48423912134661e-25, 7.470472073199248e-32, 1.4330065884610723e-38, 2.0178509043623295e-78, 7.762422313879849e-42, 1.1130262979120835e-25, 1.7294352807594737e-29, 5.081257795564035e-144, 4.604799527234278e-22, 5.780804644994604e-105, 1.0687608274077939e-16, 5.732975276393661e-79, 1.238110781902637e-39, 2.7

In [202]:
test.head(20)

Teste  Relevância  \
0   @ogustamlk gu, obrigado por nos avisar sobre i...           0   
1   @nubank nubank libera logo meu cartao de crédi...           1   
2   nubank foi o melhor cartão que eu já fiz até h...           1   
3                             @nubank quer casar cmg?           0   
4          aeeee carai, nubank aceitou meu crédito :3           1   
5   @gabeleirbag haha! como diz a música da tay. "...           0   
6   @nubank @nubank, todos esses procedimentos  nã...           1   
7   @_______jhony @fercorsato @nubank deve ser ita...           1   
8                   @nubank ja pedi um monte e nada 😭           1   
9   @malvados oi gente eu continuo precisando d do...           0   
10  @nubank já tem 1 ano esperando aumentarem meu ...           1   
11                                     @nubank tá né?           0   
12  dps de anos o nubank me manda um cartão com 50...           1   
13  @negalindaa1 por aqui nós não conseguimos veri...           0   
14  @tivor_ @nubank sim! ai o cartão precisa de um...           1   
15   @manuelajeane @junqueira_cris comenta chefinha 👀           0   
16  @ricandrademc @imaginecurve tenho quase todos ...           1   
17             @caosfanboy @nubank só se vive uma vez           0   
18  @nubank a autônoma aqui movimenta a conta todo...           1   
19                    @nubank faz teus corres, parça.           0   

    Relevância_Classificador  
0                          0  
1                          1  
2                          0  
3                          1  
4                          1  
5                          1  
6                          0  
7                          0  
8                          1  
9                          0  
10                         0  
11                         0  
12                         1  
13                         0  
14                         0  
15                         1  
16                         0  
17                         0  
18                         0  
19                         0

In [176]:
# Tweet a ser classificado
print(lista_test[0])
tweet = lista_test[0]

['@ogustamlk', 'gu', 'obrigado', 'por', 'nos', 'avisar', 'sobre', 'isso', 'como', 'não', 'podemos', 'passar', 'informações', 'sobre', 'outros', 'clientes', 'poderia', 'nos', 'chamar', 'na', 'dm', 'precisamos', 'de', 'alguns', 'dados', 'que', 'estão', 'no', 'cartão', 'pra', 'agirmos', 'mais', 'rapidamente', '😉']


In [140]:
# Probabilidade (Exercício 1 - Aula 08)
Pr = len(lista_relevante)/len(total)
Pi = len(lista_irrelevante)/len(total)
print("Probabilidade de ser relevante: {0}".format(len(lista_relevante)/len(total)))
print("Probabilidade de ser irrelevante: {0}".format(len(lista_irrelevante)/len(total)))

Probabilidade de ser relevante: 0.4029198758477986
Probabilidade de ser irrelevante: 0.5970801241522014


In [141]:
 # Usando o suavizador de Laplace
acumulador_relevante = 1
acumulador_irrelevante = 1

for h in tweet:
    
    if h not in lista_relevante:
        tabela_relevante_absoluta[h] = 0
        
    if h not in lista_irrelevante:
        tabela_irrelevante_absoluta[h] = 0
        
    acumulador_relevante *= ( tabela_relevante_absoluta[h] + 1 ) / ( len(lista_relevante) + len(total) )
    acumulador_irrelevante *= ( tabela_irrelevante_absoluta[h] + 1 ) / ( len(lista_irrelevante) + len(total) )    
    
print("Acumulador relevante = {0}".format(acumulador_relevante))
print("Acumulador irrelevante = {0}".format(acumulador_irrelevante))

Acumulador relevante = 6.755927386827296e-121
Acumulador irrelevante = 7.85659759033175e-115


In [121]:
# Probabilidade Final
print(Pr*acumulador_relevante)
print(Pi*acumulador_irrelevante)

2.7220974239371965e-121
4.691018264649167e-115


In [ ]:
# Como a segunda probabilidade é maior, o tweet tem uma maior probabilidade de ser irrelevante

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**